In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import scipy as sp
import warnings
warnings.filterwarnings('ignore')

1. Have the size of the sets changed over time?

In [2]:
df_sets = pd.read_csv("/kaggle/input/lego-database/sets.csv")
df_sets.head()

,set_num,name,year,theme_id,num_parts
0,00-1,Weetabix Castle,1970,414,471
1,0011-2,Town Mini-Figures,1978,84,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,2
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12


In [3]:
biggest_set = df_sets.sort_values('num_parts', ascending=False).iloc[0]
print(f"The biggest set is: {biggest_set['name']} with {biggest_set['num_parts']} parts.")

The biggest set is: Taj Mahal with 5922 parts.


In [4]:
df_sets_per_year = df_sets.groupby('year').agg(mean_part=('num_parts', np.mean),
                                        std=('num_parts', np.std),
                                        max_part = ('num_parts', np.max),
                                        min_part = ('num_parts', np.min),
                                        theme_nbr = ('theme_id' , pd.Series.nunique),
                                        set_nbr = ('set_num', len))

Fig_1 = px.scatter(df_sets_per_year,
                   x = df_sets_per_year.index,
                   y = 'mean_part',
                   labels = {'year':'Year',
                          'mean_part' : "Average number of part",
                          'theme_nbr' : 'Number of theme'},
                   title = "Evolution of the mean size of LEGO's set from 1950",
                   height = 600,
                   width = 1200
                  )

Fig_1.update_traces(marker=dict(size=10, color = '#0033B2')
                   )

Fig_1.show()

r, p = sp.stats.pearsonr(df_sets_per_year.index.values, df_sets_per_year['mean_part'])

if p < 0.05:
    print("The evolution of the size of LEGO's set over time is significant.")
else:
    print("There has not been significant changes in the size of LEGO's set over time.")

The evolution of the size of LEGO's set over time is significant.


In [5]:
Fig_1 = px.scatter(df_sets_per_year,
                   x = df_sets_per_year.index,
                   y = 'set_nbr',
                   color = 'theme_nbr',
                   labels = {'year':'Year',
                          'theme_nbr' : "Number of themes",
                          'set_nbr':'# sets'},
                   title = "Number of sets released per year",
                   color_continuous_scale=px.colors.sequential.Viridis,
                   height = 600,
                   width = 1200,
                   size = 'theme_nbr')


Fig_1.show()

In [6]:
Fig_1 = px.scatter(df_sets_per_year,
                   x = 'theme_nbr',
                   y = 'mean_part',
                   color = df_sets_per_year.index,
                   size = 'mean_part',
                   labels = {'set_nbr':'Number of sets',
                          'mean_part' : "Average number of parts",
                          'theme_nbr' : 'Number of themes'},
                   height = 600,
                   width = 800)


Fig_1.show()

In [7]:
df_themes = pd.read_csv("/kaggle/input/lego-database/themes.csv")
df_themes.head()

,id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0


In [8]:
df_merged = pd.merge(df_sets, df_themes, left_on = 'theme_id', right_on = 'id')
df_merged.drop(['theme_id', 'id', 'parent_id'], axis = 1, inplace = True)
df_merged.rename(columns = {'name_x' : 'set',
                            'name_y' : 'theme'}, inplace = True)
df_merged.head()

,set_num,set,year,num_parts,theme
0,00-1,Weetabix Castle,1970,471,Castle
1,0011-2,Town Mini-Figures,1978,12,Supplemental
2,0011-3,Castle 2 for 1 Bonus Offer,1987,2,Lion Knights
3,0012-1,Space Mini-Figures,1979,12,Supplemental
4,0013-1,Space Mini-Figures,1979,12,Supplemental


In [9]:
resume_theme = df_merged.groupby(['theme']).agg(mean_part = ('num_parts' , 'mean'), std = ('num_parts' , 'std'), set_nbr = ('set', len))
# Use len method rather to pd.Series.nunique to determine the number of sets per theme as some identical sets are released at different year.

top15_theme = resume_theme.sort_values(by = 'set_nbr', ascending = False)[0:15]

Fig = px.bar(top15_theme,
                 x = top15_theme['set_nbr'],
                 height = 500,
                 width = 800,
                 title = 'TOP 15 of themes based on the number of sets',
                 labels = {'theme':'Theme',
                          'mean_part' : 'Average number of parts',
                          'set_nbr' : 'Number of sets'})

Fig.show()

In [10]:
top15_bigger_theme =  resume_theme.sort_values(by = 'mean_part', ascending = False)[0:15]

Fig = px.scatter(top15_bigger_theme,
                 y = top15_bigger_theme.index,
                 x = top15_bigger_theme['mean_part'],
                 error_x = 'std',
                 color = top15_bigger_theme.index,
                 text = 'set_nbr',
                 height = 500,
                 width = 800,
                 title = 'TOP 15 of biggest themes based on the average number of parts per set',
                 labels = {'theme':'Theme',
                          'mean_part' : 'Average number of parts',
                          'set_nbr' : 'Number of sets'})

Fig.update_layout(showlegend=False)

Fig.update_traces(marker=dict(size=20))

Fig.show()

In [11]:
theme_released_year = df_merged.groupby('year')['theme'].value_counts().unstack()

Fig = px.bar(theme_released_year,
            x = theme_released_year.index,
            y = theme_released_year.columns,
            title = 'Number of sets released by year by theme',
            labels = {'value' : 'Number of sets',
                     'theme' : 'Theme',
                     'year' : 'Year'})
Fig.show()

In [12]:
fig = px.strip(df_merged,
                   x = 'year',
                   y = 'num_parts',
                   color = 'theme',
                   hover_name = 'set')

fig.show()

In [13]:
top10_theme = theme_released_year.sum().sort_values(ascending = False)[:10].index.values

Fig = px.bar(theme_released_year,
            x = theme_released_year.index,
            y = top10_theme,
            title = 'Number of sets released by year for the 10 most popular themes',
            labels = {'value' : 'Number of sets',
                     'variable' : 'Theme',
                     'year' : 'Year'})

Fig.show()

2. What colors are associated with which theme?

In [14]:
df_color = pd.read_csv("/kaggle/input/lego-database/colors.csv")
df_color['rgb'] = df_color['rgb'].apply(lambda x : '#'+x)
colors_set = dict(zip(df_color.name, df_color.rgb))
print(len(colors_set), 'colors.')

df_color.head()

135 colors.


,id,name,rgb,is_trans
0,-1,Unknown,#0033B2,f
1,0,Black,#05131D,f
2,1,Blue,#0055BF,f
3,2,Green,#237841,f
4,3,Dark Turquoise,#008F9B,f


In [15]:
df_inventories = pd.read_csv("/kaggle/input/lego-database/inventories.csv")

df_inventory_parts = pd.read_csv("/kaggle/input/lego-database/inventory_parts.csv")

theme_and_colors = pd.merge(df_merged, df_inventories, on = 'set_num')

theme_and_colors = pd.merge(theme_and_colors, df_inventory_parts, left_on = 'id', right_on = 'inventory_id')

theme_and_colors = pd.merge(theme_and_colors, df_color, left_on = 'color_id', right_on = 'id')

theme_and_colors.drop(['id_y', 'id_x'], axis = 1, inplace = True)

theme_and_colors.rename(columns = {'name' : 'color_name'}, inplace = True)
theme_and_colors.head()

,set_num,set,year,num_parts,theme,version,inventory_id,part_num,color_id,quantity,is_spare,color_name,rgb,is_trans
0,00-1,Weetabix Castle,1970,471,Castle,1,5574,29c01,4,8,f,Red,#C91A09,f
1,00-1,Weetabix Castle,1970,471,Castle,1,5574,29c01,15,6,f,White,#FFFFFF,f
2,00-1,Weetabix Castle,1970,471,Castle,1,5574,3001a,15,25,f,White,#FFFFFF,f
3,00-1,Weetabix Castle,1970,471,Castle,1,5574,3001a,4,9,f,Red,#C91A09,f
4,00-1,Weetabix Castle,1970,471,Castle,1,5574,3001a,1,4,f,Blue,#0055BF,f


In [16]:
colors_by_year = theme_and_colors.groupby('year')['color_name'].value_counts(normalize = True).unstack()

# Comprehension list to extract only colors (name and rgb) present in the dataset
colors_dict = {i:j for i,j in colors_set.items() if i in colors_by_year.columns}

Fig = px.bar(colors_by_year,
            x = colors_by_year.index,
            y = colors_by_year.columns,
            title = 'Relative distribution of colors over time',
            labels = {'value' : '%',
                     'color_name' : 'Color',
                     'year' : 'Year'},
             color_discrete_sequence=colors_by_year.columns.map(colors_dict)
            )                   

Fig.show()

In [17]:
colors_by_year

color_name,Aqua,Black,Blue,Blue-Violet,Bright Green,Bright Light Blue,Bright Light Orange,Bright Light Yellow,Bright Pink,Brown,...,Trans-Yellow,Unknown,Very Light Bluish Gray,Very Light Gray,Very Light Orange,Violet,White,Yellow,Yellowish Green,[No Color]
year,,,,,,,,,,,,,,,,,,,,,
1950,NaN,NaN,0.130435,NaN,0.043478,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.173913,0.173913,NaN,NaN
1953,NaN,NaN,0.055556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,0.166667,NaN,NaN
1954,NaN,NaN,0.132653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.285714,0.163265,NaN,NaN
1955,NaN,NaN,0.093264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.481865,0.098446,NaN,0.005181
1956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.626506,NaN,NaN,0.012048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,0.000029,0.172290,0.031863,NaN,0.005623,0.001298,0.005796,0.000807,0.005046,NaN,...,0.005046,0.000058,0.000029,NaN,NaN,NaN,0.107555,0.046107,NaN,0.004729
2014,NaN,0.177259,0.033574,NaN,0.006010,0.001183,0.007072,0.001183,0.006782,NaN,...,0.005358,0.000024,NaN,NaN,NaN,NaN,0.098163,0.040284,0.000338,0.001207
2015,NaN,0.166808,0.032235,NaN,0.004658,0.000725,0.008737,0.001159,0.005900,0.000041,...,0.004720,NaN,NaN,NaN,NaN,NaN,0.099087,0.042980,0.002236,0.000828


In [18]:
colors_by_theme = theme_and_colors.groupby('theme')['color_name'].value_counts(normalize = True).unstack()
colors_by_theme

Fig = px.bar(colors_by_theme,
            x = colors_by_theme.index,
            y = colors_by_theme.columns,
            title = 'Relative distribution of colors over time',
            labels = {'value' : '%',
                     'color_name' : 'Color',
                     'year' : 'Year'},
             color_discrete_sequence=colors_by_year.columns.map(colors_dict)
            )                   

Fig.show()